In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os

In [2]:
import math
from sklearn.metrics import mean_squared_error

# Funções Auxiliares

In [3]:
def normalize(df):
    mindf = df.min()
    maxdf = df.max()
    return (df-mindf)/(maxdf-mindf)

In [4]:
def denormalize(norm, _min, _max):
    return [(n * (_max-_min)) + _min for n in norm]

In [5]:
def split_data(df, interval):
    sample_df = df.loc[interval]

    week = (sample_df.index.day - 1) // 7 + 1

    # PARA OS TESTES:
    # 2 SEMANAS PARA TREINAMENTO
    train_df = sample_df.loc[week <= 2]

    # 1 SEMANA PARA VALIDACAO
    validation_df = sample_df.loc[week == 3]

    # 1 SEMANA PARA TESTES
    test_df = sample_df.loc[week > 3]
    
    return (train_df, validation_df, test_df)

In [6]:
def calculate_rmse(test, forecast, order, step):
    rmse = math.sqrt(mean_squared_error(test.iloc[(order):], forecast[:-step]))
    print("RMSE : "+str(rmse))
    return rmse

In [7]:
def reconstruct_ssa_series(clean, residual):
    return [r + c for r, c in zip(residual,clean)]

In [8]:
def save_obj(obj, name ):
    with open('results/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('results/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [9]:
def difference(raw_df, interval=1):
    df_diff = pd.DataFrame(columns=raw_df.columns, index=raw_df.index[1:])
    
    for col in raw_df.columns:
        raw_array = raw_df[col]
        diff = []
        for i in range(interval, len(raw_array)):
            value = raw_array[i] - raw_array[i - interval]
            diff.append(value)
        
        df_diff[col] = diff
    return df_diff

In [10]:
def inverse_difference(raw_series, diff_series):
    inverted = []
    for i in range(len(diff_series)):
        interval = len(raw_series)-i
        value = diff_series[i] + raw_series[-interval]
        inverted.append(value)
        
    return inverted

# Load Dataset
Split the data into train, validation and test subsets

In [11]:
#Set target and input variables 
target_station = 'DHHL_3'

#All neighbor stations with residual correlation greater than .90
neighbor_stations_90 = ['DHHL_3',  'DHHL_4','DHHL_5','DHHL_10','DHHL_11','DHHL_9','DHHL_2', 'DHHL_6','DHHL_7','DHHL_8']

In [12]:
df = pd.read_pickle(os.path.join(os.getcwd(), "../data/oahu/df_oahu.pkl"))

In [13]:
## Remove columns with many corrupted or missing values
df.drop(columns=['AP_1', 'AP_7'], inplace=True)

In [14]:
#Normalize Data

# Save Min-Max for Denorm
min_raw = df[target_station].min()

max_raw = df[target_station].max()

# Perform Normalization
norm_df = normalize(df)

In [15]:
# Split data
interval = ((df.index >= '2010-06') & (df.index < '2010-07'))
#interval = ((df.index >= '2010-11') & (df.index <= '2010-12'))

(train_df, validation_df, test_df) = split_data(df, interval)
(norm_train_df, norm_validation_df, norm_test_df) = split_data(norm_df, interval)

## Forecasting with Raw Time Series

For each dataset, all the time series were normalized

## FuzzyImageCNN

In [16]:
from pyFTS.partitioners import Grid, Entropy, Util as pUtil

from fts2image import FuzzyImageCNN

Using TensorFlow backend.


In [17]:
def fuzzy_cnn_forecast(train_df, test_df):

    
    _conv_layers = 2
    _dense_layer_neurons = 1024
    _dense_layers = 3
    _epochs = 30
    _filters = 8
    _kernel_size = 2
    _npartitions = 50
    _order = 8
    _pooling_size = 2
    

    fuzzy_sets = Grid.GridPartitioner(data=train_df.values, npart=_npartitions).sets
    model = FuzzyImageCNN.FuzzyImageCNN(fuzzy_sets, nlags=_order, steps=1,
                                       conv_layers=_conv_layers, dense_layers = _dense_layers, 
                                        dense_layer_neurons=_dense_layer_neurons, filters=_filters,
                                       kernel_size=_kernel_size, pooling_size=_pooling_size)
    model.fit(train_df, epochs=_epochs, batch_size=64)

    forecast = model.predict(test_df)    

    return forecast

In [21]:
steps = 1

forecast = fuzzy_cnn_forecast(norm_train_df[target_station], norm_test_df[target_station])
forecast = denormalize(forecast, min_raw, max_raw)

Epoch 1/1
846/846 [==============================] - 1s 1ms/step - loss: 0.0331


In [22]:
_order = 8

In [23]:
forecast.append(0) ## para manter o mesmo tamanho dos demais
rmse = calculate_rmse(test_df[target_station], forecast, _order, steps)

RMSE : 148.81590526087942


### Parameter optimization

In [18]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt import space_eval

In [36]:
space = {'npartitions': hp.choice('npartitions', [50, 100, 150, 200]),
            'order': hp.choice('order', list(np.arange(4,10))),
             'epochs': hp.choice('epochs', [30, 50, 100])}

In [37]:
def cnn_objective(params):
    print(params)
    forecast = fuzzy_cnn_forecast(norm_train_df[target_station], norm_test_df[target_station], params['order'], 1, params['npartitions'], params['epochs'])
    forecast = denormalize(forecast, min_raw, max_raw)
    forecast.append(0) ## para manter o mesmo tamanho dos demais
    rmse = calculate_rmse(test_df[target_station], forecast, params['order'], 1) 
    return {'loss': rmse, 'status': STATUS_OK}

In [38]:
trials = Trials()
best = fmin(cnn_objective, space, algo=tpe.suggest, max_evals =200, trials=trials)

{'epochs': 50, 'npartitions': 50, 'order': 8}
Epoch 1/1
846/846 [==============================] - 2s 2ms/step - loss: 0.0626
RMSE : 223.6843606526732
{'epochs': 100, 'npartitions': 100, 'order': 6}
Epoch 1/1
848/848 [==============================] - 2s 2ms/step - loss: 0.0867
RMSE : 191.90993566596345
{'epochs': 100, 'npartitions': 50, 'order': 4}
Epoch 1/1
850/850 [==============================] - 1s 2ms/step - loss: 0.0928
RMSE : 266.834131922861
{'epochs': 30, 'npartitions': 150, 'order': 5}
Epoch 1/1
849/849 [==============================] - 2s 2ms/step - loss: 0.0373
RMSE : 169.86460110128886
{'epochs': 30, 'npartitions': 50, 'order': 7}
Epoch 1/1
847/847 [==============================] - 2s 2ms/step - loss: 0.0388
RMSE : 185.52169762433465
{'epochs': 50, 'npartitions': 200, 'order': 8}
Epoch 1/1
846/846 [==============================] - 3s 3ms/step - loss: 0.0359
RMSE : 179.8822960193808
{'epochs': 100, 'npartitions': 150, 'order': 8}
Epoch 1/1
846/846 [====================

In [39]:
print('best: ')
print(space_eval(space, best))

best: 
{'epochs': 30, 'npartitions': 50, 'order': 4}


## Persistence

In [29]:
def persistence_forecast(train, test, step):
    predictions = []
    
    for t in np.arange(0,len(test), step):
        yhat = [test.iloc[t]]  * step
        predictions.extend(yhat)
        
    return predictions

In [30]:
step = 1
persistence_order = 1

forecast = persistence_forecast(norm_train_df[target_station], norm_test_df[target_station],step)
forecast = denormalize(forecast, min_raw, max_raw)

rmse = calculate_rmse(test_df[target_station], forecast, persistence_order, step)

RMSE : 126.5116260261361


In [ ]:
result = {'rmse': rmse, 'final': forecast}
save_obj(result, name="oahu_raw_persistence_1")

## Long Short Term Memory - LSTM

In [40]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [41]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [42]:
def lstm_multi_forecast(train_df, test_df, _order, _steps, _neurons, _epochs):

    
    nfeat = len(train_df.columns)
    nlags = _order
    nsteps = _steps
    nobs = nlags * nfeat
    
    train_reshaped_df = series_to_supervised(train_df, n_in=nlags, n_out=nsteps)
    train_X, train_Y = train_reshaped_df.iloc[:,:nobs].values, train_reshaped_df.iloc[:,-nfeat].values
    train_X = train_X.reshape((train_X.shape[0], nlags, nfeat))
    
    test_reshaped_df = series_to_supervised(test_df, n_in=nlags, n_out=nsteps)
    test_X, test_Y = test_reshaped_df.iloc[:,:nobs].values, test_reshaped_df.iloc[:,-nfeat].values
    test_X = test_X.reshape((test_X.shape[0], nlags, nfeat))
    
    # design network
    model = Sequential()
    model.add(LSTM(_neurons, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam')
    
    # fit network
    model.fit(train_X, train_Y, epochs=_epochs, batch_size=72, verbose=False, shuffle=False)
    
    forecast = model.predict(test_X)
        
    return forecast

In [43]:
neurons = 50
lstm_order = 2
epochs = 100
steps = 1

forecast = lstm_multi_forecast(norm_train_df[[target_station]], norm_test_df[[target_station]], lstm_order, steps, neurons, epochs)
forecast = denormalize(forecast, min_raw, max_raw)

In [44]:
forecast.append(0) ## para manter o mesmo tamanho dos demais

In [46]:
rmse = calculate_rmse(test_df[target_station], forecast, lstm_order, steps)

RMSE : 123.35325991355555


In [ ]:
result = {'rmse': rmse, 'final': forecast}
save_obj(result, name="oahu_raw_lstm_multi_1")

## Multi Layer Perceptron - MLP

In [ ]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
def mlp_forecast(train_df, test_df, _order, _steps, _neurons, _epochs):

    
    nfeat = len(train_df.columns)
    nlags = _order
    nsteps = _steps
    nobs = nlags * nfeat
    
    train_reshaped_df = series_to_supervised(train_df, n_in=nlags, n_out=nsteps)
    train_X, train_Y = train_reshaped_df.iloc[:,:nobs].values, train_reshaped_df.iloc[:,-nfeat].values
    
    test_reshaped_df = series_to_supervised(test_df, n_in=nlags, n_out=nsteps)
    test_X, test_Y = test_reshaped_df.iloc[:,:nobs].values, test_reshaped_df.iloc[:,-nfeat].values
    
    # design network
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_dim=train_X.shape[1]))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')

    # fit network
    history = model.fit(train_X, train_Y, epochs=_epochs, batch_size=72, verbose=False, shuffle=False)   

    forecast = model.predict(test_X)
        
    return forecast

In [ ]:
neurons = 50
mlp_order = 4
epochs = 500
steps = 1

forecast = mlp_forecast(norm_train_df[[target_station]], norm_test_df[[target_station]], mlp_order, steps, neurons, epochs)
forecast = denormalize(forecast, min_raw, max_raw)

forecast.append(0) ## para manter o mesmo tamanho dos demais

rmse = calculate_rmse(test_df[target_station], forecast, mlp_order, steps)

In [ ]:
result = {'rmse': rmse, 'final': forecast}
save_obj(result, name="oahu_raw_mlp_uni_1")